In [ ]:
# Zotero MCP - Create Note 功能演示

这个 notebook 演示了如何使用 Zotero MCP 中的 `zotero_create_note` 功能。我们将展示各种使用场景，从简单的笔记创建到更复杂的应用场景。


In [ ]:
## 环境设置

首先，我们需要设置必要的环境变量和导入必要的库。


In [ ]:
import os
import json
import requests
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()

# 设置 MCP 服务器 URL
MCP_SERVER_URL = os.getenv("MCP_SERVER_URL", "http://localhost:8000")


In [ ]:
## 定义辅助函数

下面我们定义一些辅助函数，用于与 MCP 服务器通信并调用 Zotero 工具。


In [ ]:
def call_mcp_tool(tool_name, arguments):
    """调用 MCP 工具的通用函数"""
    payload = {
        "tool": tool_name,
        "arguments": arguments
    }
    
    response = requests.post(
        f"{MCP_SERVER_URL}/tools",
        json=payload,
        headers={"Content-Type": "application/json"}
    )
    
    return response.json()

def create_note(item_key, note_title, note_text, tags=None):
    """创建一条 Zotero 笔记"""
    arguments = {
        "item_key": item_key,
        "note_title": note_title,
        "note_text": note_text
    }
    
    if tags:
        arguments["tags"] = tags
    
    return call_mcp_tool("zotero_create_note", arguments)

def search_items(query, limit=5):
    """搜索 Zotero 条目"""
    return call_mcp_tool("zotero_search_items", {"query": query, "limit": limit})


In [ ]:
## 验证 MCP 服务器连接

在继续之前，让我们先验证 MCP 服务器是否正在运行并且可以连接。


In [ ]:
try:
    # 尝试获取最近的条目，仅用于测试连接
    response = call_mcp_tool("zotero_get_recent", {"limit": 1})
    print("MCP 服务器连接成功")
    print("API 响应:")
    print(json.dumps(response, indent=2, ensure_ascii=False)[:500] + "...")
except Exception as e:
    print(f"错误: 无法连接到 MCP 服务器: {e}")
    print("请确保 MCP 服务器正在运行，并且可以在配置的 URL 访问。")


In [ ]:
## 1. 查找条目

在创建笔记之前，我们需要知道要为哪个条目创建笔记。让我们先搜索一些条目。


In [ ]:
# 搜索与"机器学习"相关的条目
search_results = search_items("机器学习")
print(search_results.get("result", "未找到任何条目"))


In [ ]:
在上面的结果中找到一个条目的键，我们将使用它来创建笔记。

如果您没有看到任何条目，请尝试搜索其他关键词或直接输入您知道的条目键。


In [ ]:
# 设置您要使用的条目键
target_item_key = "YOUR_ITEM_KEY_HERE"  # 替换为您的条目键

# 验证条目存在并获取详细信息
item_details = call_mcp_tool("zotero_get_item_metadata", {"item_key": target_item_key})
print(item_details.get("result", "未找到条目"))


In [ ]:
## 2. 创建简单笔记

现在我们将为选定的条目创建一条简单的笔记。


In [ ]:
# 创建简单的纯文本笔记
note_title = "简单笔记示例"
note_text = "这是一个简单的笔记。\n\n它包含两个段落。"
tags = ["示例", "MCP"]

result = create_note(target_item_key, note_title, note_text, tags)
print(result.get("result", "创建笔记失败"))


In [ ]:
## 3. 创建格式化的笔记

接下来，我们将创建一个包含丰富 HTML 格式的笔记。


In [ ]:
# 创建富文本格式笔记
formatted_note_title = "格式化笔记示例"
formatted_note_text = """
<h1>文献研究笔记</h1>
<p><strong>核心观点</strong>：这篇文献提出了一种新的方法来解决特定问题。</p>

<h2>主要贡献</h2>
<ol>
  <li>提出了新算法 X，比现有方法提高了性能</li>
  <li>证明了理论 Y 在特定条件下的适用性</li>
  <li>提供了大规模实验验证</li>
</ol>

<h2>关键引用</h2>
<ul>
  <li>Smith et al. (2020) - 基础理论</li>
  <li>Zhang and Wang (2021) - 比较方法</li>
</ul>

<p><em>待办事项</em>: 阅读参考文献 #3 和 #7</p>
"""

tags = ["研究", "重要", "待跟进"]

result = create_note(target_item_key, formatted_note_title, formatted_note_text, tags)
print(result.get("result", "创建笔记失败"))


In [ ]:
## 4. 使用 AI 生成的内容创建笔记

在这个示例中，我们将演示如何使用 AI（如大型语言模型）生成笔记内容，然后通过 MCP 保存到 Zotero。

注意：此示例假设您有访问 OpenAI API 的权限，并且已经安装了必要的库。


In [ ]:
# 仅在有 OpenAI 库时执行
try:
    import openai
    
    # 设置 OpenAI API 密钥
    openai.api_key = os.getenv("OPENAI_API_KEY")
    
    # 如果已经设置了 API 密钥，继续执行；否则跳过
    if openai.api_key:
        # 获取条目信息作为 AI 的输入
        item_info = item_details.get("result", "")
        
        if item_info:
            # 调用 OpenAI API
            response = openai.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "您是一位学术研究助手，专长于提取和总结学术文献的核心观点和贡献。"},
                    {"role": "user", "content": f"基于以下文献信息，生成一个结构化的研究笔记，包括：1) 主要研究问题，2) 核心方法，3) 主要发现，4) 可能的研究局限性，5) 未来研究方向。请使用HTML格式，添加适当的标题和列表。\n\n{item_info}"}
                ]
            )
            
            # 提取 AI 生成的笔记内容
            ai_note_content = response.choices[0].message.content
            
            # 创建笔记
            ai_note_title = "AI生成的文献笔记"
            result = create_note(target_item_key, ai_note_title, ai_note_content, ["AI生成", "自动笔记"])
            print(result.get("result", "创建笔记失败"))
            
            # 显示 AI 生成的内容
            print("\nAI 生成的笔记内容预览:\n")
            print(ai_note_content[:500] + "...")
        else:
            print("无法获取条目信息作为 AI 输入")
    else:
        print("未设置 OpenAI API 密钥，跳过 AI 笔记生成示例")
except ImportError:
    print("未安装 openai 库，跳过 AI 笔记生成示例")
except Exception as e:
    print(f"AI 笔记生成过程中出错: {str(e)}")


In [ ]:
## 5. 批量创建笔记

下面我们演示如何批量为多个条目创建笔记。


In [ ]:
# 模拟批量处理数据
batch_notes = [
    {
        "item_key": target_item_key,  # 使用前面找到的条目键，在实际应用中应该是不同的键
        "title": "批量笔记 1",
        "text": "这是批量创建的第一条笔记。",
        "tags": ["批量", "组1"]
    },
    {
        "item_key": target_item_key,  # 在实际应用中应该是不同的键
        "title": "批量笔记 2",
        "text": "这是批量创建的第二条笔记。",
        "tags": ["批量", "组2"]
    }
]

# 批量处理
results = []
for note_data in batch_notes:
    result = create_note(
        note_data["item_key"],
        note_data["title"],
        note_data["text"],
        note_data["tags"]
    )
    results.append(result)
    print(f"处理 {note_data['title']}: {result.get('result', '失败')}")


In [ ]:
## 6. 从已有笔记中导入内容

这个示例演示如何从现有文件导入内容，并创建 Zotero 笔记。


In [ ]:
# 创建一个示例笔记文件
with open("sample_note.md", "w") as f:
    f.write("""
# 导入的研究笔记

这是一个从外部文件导入的笔记示例。

## 关键点

- 点1: 重要观察
- 点2: 值得注意的数据
- 点3: 相关结论

## 参考信息

页码: 145-148
章节: 第7章
""")


In [ ]:
# 从文件导入笔记内容
try:
    from markdown2 import Markdown
    
    # 读取 Markdown 文件
    with open("sample_note.md", "r") as f:
        md_content = f.read()
    
    # 将 Markdown 转换为 HTML
    markdowner = Markdown()
    html_content = markdowner.convert(md_content)
    
    # 创建笔记
    imported_note_title = "导入的文件笔记"
    result = create_note(target_item_key, imported_note_title, html_content, ["导入", "外部源"])
    print(result.get("result", "创建笔记失败"))
except ImportError:
    print("未安装 markdown2 库，使用普通文本导入")
    
    # 使用普通文本
    with open("sample_note.md", "r") as f:
        text_content = f.read()
    
    result = create_note(target_item_key, "导入的文本文件笔记", text_content, ["导入", "外部源"])
    print(result.get("result", "创建笔记失败"))


In [ ]:
## 7. 错误处理示例

下面我们演示如何处理常见的错误情况。


In [ ]:
# 尝试为不存在的条目创建笔记
invalid_key = "INVALID_ITEM_KEY_12345"
result = create_note(invalid_key, "错误测试笔记", "此笔记不应该创建成功")
print(f"使用无效条目键的结果: {result.get('result', '未知错误')}")


In [ ]:
## 总结

在本演示中，我们展示了如何使用 Zotero MCP 中的 `zotero_create_note` 功能来：

1. 创建简单的纯文本笔记
2. 创建富 HTML 格式的笔记
3. 使用 AI 生成笔记内容（如果有 OpenAI API 访问权限）
4. 批量创建多个笔记
5. 从外部文件导入笔记内容
6. 处理常见错误情况

这些示例应该能帮助您在自己的应用程序中集成 Zotero 笔记创建功能。
